# Setting Node Capacities

Capacities of a FABRIC node are basic characteristics of the virtual machine including number of compute core, amount of memory, and amount of local disk.  This notebook will demonstrate the options for setting these node capciites.

## Configure the Environment

In [ ]:
import os
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
import json

In [ ]:
ssh_key_file_priv=os.environ['HOME']+"/.ssh/id_rsa"
ssh_key_file_pub=os.environ['HOME']+"/.ssh/id_rsa.pub"

ssh_key_pub = None
with open (ssh_key_file_pub, "r") as myfile:
    ssh_key_pub=myfile.read()
    ssh_key_pub=ssh_key_pub.strip()

In [ ]:
credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
print(f"FABRIC Credential Manager   : {credmgr_host}")

orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"FABRIC Orchestrator         : {orchestrator_host}")

## Create Slice Manager Object

In [ ]:
slice_manager = SliceManager(oc_host=orchestrator_host, 
                             cm_host=credmgr_host ,
                             project_name='all', 
                             scope='all')

# Initialize the slice manager
slice_manager.initialize()

## Configure Slice Parameters



In [ ]:
slice_name='MySlice'
node_name='node1'
site='RENC'
image_name='default_centos_8'
image_type='qcow2'

## Setting Capacities

We are going to be creating slices that contain one node each.

We need to specify the resources (number of cores, amount of ram and amount of disk space) that we want to allocate to our node.

We can do that in two ways:
 - Using Capacities()
 - Using capacity hints.

## Example 1: Exact Capacities

Let's create our first slice that contains one node. We will use `Capacities()` to specify the resources that we want to allocate.

The line `cap.set_fields(core=2, ram=8, disk=10)` specifies that we want to reserve a node with 2 cores, 8GB of RAM and 10GB of disk.

In [ ]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType
# Create topology
t = ExperimentTopology()

# Add node
n1 = t.add_node(name=node_name, site=site)

# Set capacities
cap = Capacities()
cap.set_fields(core=2, ram=8, disk=10)

# Set Properties
n1.set_properties(capacities=cap, image_type=image_type, image_ref=image_name)

# Generate Slice Graph
slice_graph = t.serialize()

# Request slice from Orchestrator
return_status, slice_reservations = slice_manager.create(slice_name=slice_name, slice_graph=slice_graph, ssh_key=ssh_key_pub)

if return_status == Status.OK:
    slice_id = slice_reservations[0].get_slice_id()
    print("Submitted slice creation request. Slice ID: {}".format(slice_id))
else:
    print(f"Failure: {slice_reservations}")

Get the slice and topology

In [ ]:
import time
def wait_for_slice(slice,timeout=180,interval=10,progress=False):
    timeout_start = time.time()

    if progress: print("Waiting for slice .", end = '')
    while time.time() < timeout_start + timeout:
        return_status, slices = slice_manager.slices(excludes=[SliceState.Dead])

        if return_status == Status.OK:
            slice = list(filter(lambda x: x.slice_name == slice_name, slices))[0]
            if slice.slice_state == "StableOK":
                if progress: print(" Slice state: {}".format(slice.slice_state))
                return slice
            if slice.slice_state == "Closing" or slice.slice_state == "Dead":
                if progress: print(" Slice state: {}".format(slice.slice_state))
                return slice    
        else:
            print(f"Failure: {slices}")
        
        if progress: print(".", end = '')
        time.sleep(interval)
    
    if time.time() >= timeout_start + timeout:
        if progress: print(" Timeout exceeded ({} sec). Slice: {} ({})".format(timeout,slice.slice_name,slice.slice_state))
        return slice    



return_status, slices = slice_manager.slices(excludes=[SliceState.Dead,SliceState.Closing])

if return_status == Status.OK:
    slice = list(filter(lambda x: x.slice_name == slice_name, slices))[0]
    slice = wait_for_slice(slice, progress=True)
    return_status, experiment_topology = slice_manager.get_slice_topology(slice_object=slice)


Print the allocated capacities

In [ ]:
for node_name, node in experiment_topology.nodes.items():
    print("Node {}:  ".format(node.name))
    print("   Cores             : {}".format(node.get_property(pname='capacity_allocations').core))
    print("   RAM               : {}".format(node.get_property(pname='capacity_allocations').ram))
    print("   Disk              : {}".format(node.get_property(pname='capacity_allocations').disk))

It says that our node has 2 cores, 8GB of RAM and 10GB of disk space, which is what we requested.

Now let's delete the slice.

In [ ]:
return_status, result = slice_manager.delete(slice_object=slice)
print("Response Status {}".format(return_status))

## Example 2: Rounded Capacities

Now let's try something else. Let's try to request 2 cores, 8GB of RAM, and 50GB of disk space.

Again, we are going to use `cap.set_fields(core=2, ram=8, disk=50)`.

In [ ]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType
# Create topology
t = ExperimentTopology()

# Add node
n1 = t.add_node(name=node_name, site=site)

# Set capacities
cap = Capacities()
cap.set_fields(core=2, ram=8, disk=50)

# Set Properties
n1.set_properties(capacities=cap, image_type=image_type, image_ref=image_name)

# Generate Slice Graph
slice_graph = t.serialize()

# Request slice from Orchestrator
return_status, slice_reservations = slice_manager.create(slice_name=slice_name, slice_graph=slice_graph, ssh_key=ssh_key_pub)

if return_status == Status.OK:
    slice_id = slice_reservations[0].get_slice_id()
    print("Submitted slice creation request. Slice ID: {}".format(slice_id))
else:
    print(f"Failure: {slice_reservations}")

Get the slice and topology

In [ ]:
return_status, slices = slice_manager.slices(excludes=[SliceState.Dead,SliceState.Closing])

if return_status == Status.OK:
    slice = list(filter(lambda x: x.slice_name == slice_name, slices))[0]
    slice = wait_for_slice(slice, progress=True)
    return_status, experiment_topology = slice_manager.get_slice_topology(slice_object=slice)


Print the allocated capacities

In [ ]:
for node_name, node in experiment_topology.nodes.items():
    print("Node {}:  ".format(node.name))
    print("   Cores             : {}".format(node.get_property(pname='capacity_allocations').core))
    print("   RAM               : {}".format(node.get_property(pname='capacity_allocations').ram))
    print("   Disk              : {}".format(node.get_property(pname='capacity_allocations').disk))

We can see that we were allocated 2 cores, 8GB of ram, but 100GB of disk space instead of 50GB.

The reason for this is that we have discrete "capacity hints". The node can only be an instance of one of those capacity hints.

See the very last cell in this notebook for the complete list of available capacity hints.

This is an exerpt of the available capacity hints. Full list available [here](https://github.com/fabric-testbed/InformationModel/blob/master/fim/slivers/data/instance_sizes.json).

    "fabric.c16.m64.d10": {"core":16, "ram":64, "disk": 10},
    "fabric.c32.m128.d10": {"core":32, "ram":128, "disk": 10},
    "fabric.c1.m4.d100": {"core":1, "ram":4, "disk": 100},
    "fabric.c2.m8.d100": {"core":2, "ram":8, "disk": 100},
    "fabric.c4.m16.d100": {"core":4, "ram":16, "disk": 100},

We can see that the disk space can only be 10GB or 100GB. So when we requested 50GB, it was rounded up to 100GB.

### Now let's delete the slice.

In [ ]:
return_status, result = slice_manager.delete(slice_object=slice)

print("Response Status {}".format(return_status))

## Example 3: Capacity Hints

Finally, we can directly set the resources that we need using a "capacity hint" string. _Please see the very last cell in this notebook for the complete list of available capacity hints._

We can set the needed resources like so:

`capacity_hints=CapacityHints().set_fields(instance_type='fabric.c2.m8.d10')`.

This would reserve a node with 2 processor cores, 8GB of memory and 10GB of disk space.
 - The number next to the `c` is the number of cores.
 - The number next to the `m` is the amount of memory in GB.
 - The number next to the `d` is the amount of disk space in GB.

We can pick any capacity hint string from the list.

In [ ]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType, CapacityHints
# Create topology
t = ExperimentTopology()

# Add node
n1 = t.add_node(name=node_name, site=site)

# Set capacities
cap = Capacities()
cap.set_fields(core=2, ram=8, disk=50)

# Set Properties
n1.set_properties(capacities=cap, image_type=image_type, image_ref=image_name)

# Set Properties
n1.set_properties(capacity_hints=CapacityHints().set_fields(instance_type='fabric.c2.m8.d10'),
                  image_type=image_type, 
                  image_ref=image_name)


# Generate Slice Graph
slice_graph = t.serialize()

# Request slice from Orchestrator
return_status, slice_reservations = slice_manager.create(slice_name=slice_name, slice_graph=slice_graph, ssh_key=ssh_key_pub)

if return_status == Status.OK:
    slice_id = slice_reservations[0].get_slice_id()
    print("Submitted slice creation request. Slice ID: {}".format(slice_id))
else:
    print(f"Failure: {slice_reservations}")

Get the slice and topology

In [ ]:
return_status, slices = slice_manager.slices(excludes=[SliceState.Dead,SliceState.Closing])

if return_status == Status.OK:
    slice = list(filter(lambda x: x.slice_name == slice_name, slices))[0]
    return_status, experiment_topology = slice_manager.get_slice_topology(slice_object=slice)
    slice = wait_for_slice(slice, progress=True)

Print the allocated capacities

In [ ]:
for node_name, node in experiment_topology.nodes.items():
    print("Node {}:  ".format(node.name))
    print("   Cores             : {}".format(node.get_property(pname='capacity_allocations').core))
    print("   RAM               : {}".format(node.get_property(pname='capacity_allocations').ram))
    print("   Disk              : {}".format(node.get_property(pname='capacity_allocations').disk))

We can see that we got the resources that we requested.

Now let's delete the slice.

In [ ]:
return_status, result = slice_manager.delete(slice_object=slice)

print("Response Status {}".format(return_status))
print("Response received {}".format(result))

## Capacity hints (and their descriptions) below. 

Full list available [here](https://github.com/fabric-testbed/InformationModel/blob/master/fim/slivers/data/instance_sizes.json).

    {
      "fabric.c1.m4.d10": {"core":1, "ram":4, "disk": 10},
      "fabric.c2.m8.d10": {"core":2, "ram":8, "disk": 10},
      "fabric.c4.m16.d10": {"core":4, "ram":16, "disk": 10},
      "fabric.c8.m32.d10": {"core":8, "ram":32, "disk": 10},
      "fabric.c16.m64.d10": {"core":16, "ram":64, "disk": 10},
      "fabric.c32.m128.d10": {"core":32, "ram":128, "disk": 10},
      "fabric.c1.m4.d100": {"core":1, "ram":4, "disk": 100},
      "fabric.c2.m8.d100": {"core":2, "ram":8, "disk": 100},
      "fabric.c4.m16.d100": {"core":4, "ram":16, "disk": 100},
      "fabric.c8.m32.d100": {"core":8, "ram":32, "disk": 100},
      "fabric.c16.m64.d100": {"core":16, "ram":64, "disk": 100},
      "fabric.c32.m128.d100": {"core":32, "ram":128, "disk": 100},
      "fabric.c1.m4.d500": {"core":1, "ram":4, "disk": 500},
      "fabric.c2.m8.d500": {"core":2, "ram":8, "disk": 500},
      "fabric.c4.m16.d500": {"core":4, "ram":16, "disk": 500},
      "fabric.c8.m32.d500": {"core":8, "ram":32, "disk": 500},
      "fabric.c16.m64.d500": {"core":16, "ram":64, "disk": 500},
      "fabric.c32.m128.d500": {"core":32, "ram":128, "disk": 500},
      "fabric.c1.m4.d2000": {"core":1, "ram":4, "disk": 2000},
      "fabric.c2.m8.d2000": {"core":2, "ram":8, "disk": 2000},
      "fabric.c4.m16.d2000": {"core":4, "ram":16, "disk": 2000},
      "fabric.c8.m32.d2000": {"core":8, "ram":32, "disk": 2000},
      "fabric.c16.m64.d2000": {"core":16, "ram":64, "disk": 2000},
      "fabric.c32.m128.d2000": {"core":32, "ram":128, "disk": 2000},
      "fabric.c64.m384.d4000": {"core":64, "ram":384, "disk": 4000}
    }